In [ ]:
import numpy as np
from pathlib import Path


dataset_name = 'ml-1m'
group_num = 5
batch_size = 2048

train = np.load(f'../../data/{dataset_name}/point/train.npy',allow_pickle=True) 
item_freq = np.load(f'../../data/{dataset_name}/point/item_freq.npy',allow_pickle=True)
user_freq = np.load(f'../../data/{dataset_name}/point/user_freq.npy',allow_pickle=True)

candidate_u = np.load(f'../../data/{dataset_name}/point/'+str(group_num)+'/candidate_u.npy',allow_pickle=True).tolist()
pos_u = np.load(f'../../data/{dataset_name}/point/'+str(group_num)+'/pos_u.npy',allow_pickle=True).tolist()
print('data loaded!')
pos_train = train
path = Path(f'../../data/{dataset_name}/point')
path.mkdir(parents=True, exist_ok=True)
        
num_users = user_freq.shape[0]
num_items = item_freq.shape[0]

neg_num = 4 
n_batch = num_users//batch_size
user_matrix = np.zeros((num_users,neg_num))

for u in range(num_users):
    for i in range(neg_num):
        user_matrix[u][i] = u

user_all = np.arange(num_users).astype(np.int32)
np.random.shuffle(user_all)

user_batch = []
train_batch = []
pos_cnt = np.zeros(num_users,)
for j in range(n_batch):
    user_batch.append(user_all[j*batch_size:(j+1)*batch_size])
    train_ = np.zeros((0,2))
    #train_neg = np.zeros((0,3))
    for u in user_batch[j]:
        train_ = np.r_[train_,pos_train[pos_train[:, 0]==u]]
        pos_cnt[u] = np.sum(pos_train[:, 0]==u)
        #train_neg = np.r_[train_neg,]
    train_batch.append(train_)

user_batch.append(user_all[n_batch*batch_size:num_users])
train_ = np.zeros((0,2))
for u in user_batch[n_batch]:
    train_ = np.r_[train_,pos_train[pos_train[:, 0]==u]]
    pos_cnt[u] = np.sum(pos_train[:, 0]==u)
train_batch.append(train_)

pos_cnt =  pos_cnt.astype(np.int)


eval_mask= []

for j in range(n_batch+1):
    user_cnt = user_batch[j].shape[0]
    mask_ = np.zeros((user_cnt,num_items))
    for u in range(user_cnt):
        mask_[u][candidate_u[user_batch[j][u]]]=1
    eval_mask.append(mask_)

np.save(str(path/str(group_num)/'eval_mask.npy'),arr=np.array(eval_mask))

np.save(str(path/str(group_num)/'train_batch.npy'),arr=np.array(train_batch))
np.save(str(path/str(group_num)/'user_batch.npy'),arr=np.array(user_batch))
np.save(str(path/str(group_num)/'pos_cnt.npy'),arr=np.array(pos_cnt))


max_iters = 201
train_batch_neg_list = [[[]for j in range(n_batch+1)] for i in range(max_iters)]

allitem = set(np.arange(num_items))
can = [[]for j in range(num_users)]
for u in range(num_users):
    can[u] = list(allitem - set(pos_u[u]))
    #print(can[u])
    

for i in range(max_iters):
    if i%20 ==0:
        print(i)
    for j in range(n_batch+1):
        train_batch_neg = np.zeros((0,2))
        for u in user_batch[j]:
            u_neg = np.zeros((pos_cnt[u]*neg_num,2))
            u_neg[:,0] = u
            u_neg[:,1] = np.random.choice(can[u], size=np.int(neg_num*pos_cnt[u]))
            train_batch_neg = np.r_[train_batch_neg,u_neg]
        train_batch_neg_list[i][j]=train_batch_neg

np.save(str(path/str(group_num)/'train_batch_neg_list.npy'),arr=np.array(train_batch_neg_list))
print(dataset_name,'Data Presampling Done!')